In [71]:
import os
import shutil

import numpy as np
import pandas as pd

from pathlib import Path
from typing import List

from tqdm import tqdm

In [72]:
annotations_path = Path('..') / '..' / '..' / 'storage' / 'data' / 'annotations.csv'
texts_path = Path('..') / '..' / '..' / 'storage' / 'data' / 'data.csv'

annotations_df = pd.read_csv(annotations_path, 
                             sep=',', 
                             dtype={'text_id': np.int32, 'user_id': np.int32, 'Pozytywne': np.int32, 'Negatywne': np.int32, 'Radość': np.int32, 
                                    'Zachwyt': np.int32, 'Inspiruje': np.int32, 'Spokój': np.int32, 'Zaskoczenie': np.int32, 'Współczucie': np.int32, 
                                    'Strach': np.int32, 'Smutek': np.int32, 'Wstręt': np.int32, 'Złość': np.int32, 'Ironiczny': np.int32, 
                                    'Żenujący': np.int32, 'Wulgarny': np.int32, 'Polityczny': np.int32, 'Interesujący': np.int32, 
                                    'Zrozumiały': np.int32, 'Zgadzam się z tekstem': np.int32, 'Wierzę w tę informację': np.int32, 
                                    'Potrzebuję więcej informacji, aby ocenić ten tekst': np.int32, 'Czuję sympatię do autora': np.int32, 
                                    'Obraża mnie': np.int32, 'Może kogoś atakować / obrażać / lekceważyć': np.int32, 'Mnie bawi/śmieszy?': np.int32, 
                                    'Może kogoś bawić?': np.int32})

texts_df = pd.read_csv(texts_path, sep=',', dtype={'text_id': np.int32, 'text': str})

annotations_df

,text_id,user_id,Pozytywne,Negatywne,Radość,Zachwyt,Inspiruje,Spokój,Zaskoczenie,Współczucie,...,Zrozumiały,Zgadzam się z tekstem,Wierzę w tę informację,"Potrzebuję więcej informacji, aby ocenić ten tekst",Czuję sympatię do autora,Obraża mnie,Może kogoś atakować / obrażać / lekceważyć,Mnie bawi/śmieszy?,Może kogoś bawić?,updated_at
0,0,25,0,2,0,0,0,0,0,2,...,10,1,3,0,0,0,8,-1,2,2022-12-21 17:31:49.631285+00
1,0,4,8,3,0,6,8,3,2,0,...,4,7,3,7,0,0,8,0,0,2022-12-06 20:37:25.096306+00
2,0,24,6,8,3,2,0,0,4,8,...,7,3,3,1,2,6,7,0,0,2022-12-07 10:38:29.634315+00
3,0,33,2,8,2,2,2,2,6,7,...,2,0,0,7,0,8,8,0,0,2022-12-11 22:04:28.854421+00
4,0,6,5,5,6,3,3,3,0,0,...,10,7,9,0,3,0,3,0,0,2022-12-20 16:19:10.727938+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31630,999,36,0,3,0,0,0,3,5,6,...,9,2,8,4,-1,2,3,0,0,2023-02-09 21:55:19.529524+00
31631,999,17,3,0,0,0,0,4,0,0,...,10,10,10,0,10,0,0,0,0,2023-02-07 11:45:56.639493+00
31632,999,18,6,0,0,0,0,0,0,0,...,8,10,10,0,8,0,0,0,0,2023-02-20 07:06:56.124013+00
31633,999,10,3,1,4,9,7,5,2,0,...,7,9,9,0,10,0,0,0,0,2023-02-02 14:41:27.420875+00


In [73]:
# merge annotations with texts
data_df = pd.merge(annotations_df, texts_df, on='text_id')

In [74]:
user_id = 15
data_df_user = data_df[data_df['user_id'] == user_id]
texts_to_check = pd.DataFrame(columns=['text', 'updated_at_0', 'updated_at_1'])

working_dir = Path('..', '..', '..', 'storage', 'doccano', '2023_ecai_nlperspectives', 'timestamp_extraction')

if not os.path.exists(working_dir):
    os.makedirs(working_dir)

with open(working_dir / 'texts.csv') as f:
    for line in f:
        text = line.strip()
        timestamp_list = data_df_user[data_df_user['text'] == text]['updated_at'].tolist()
        timestamp_list = sorted(timestamp_list)
        if text == '\n' or text == '':
            updated_at_0 = ''
            updated_at_1 = ''
        elif len(timestamp_list) == 0:
            print(text)
            updated_at_0 = ''
            updated_at_1 = ''
        elif len(timestamp_list) == 1:
            updated_at_0 = timestamp_list[0]
            updated_at_1 = timestamp_list[0] 
        else:
            updated_at_0 = timestamp_list[0]
            updated_at_1 = timestamp_list[1] 
        
        new_row = pd.DataFrame.from_dict({'text': [text], 'updated_at_0': [updated_at_0], 'updated_at_1': [updated_at_1]})
        texts_to_check = pd.concat([texts_to_check, new_row], axis=0)


,text,updated_at_0,updated_at_1
0,"„A gdy Baranek otworzył pieczęć piątą, ujrzałe...",2023-01-12 09:39:49.563867+00,2023-01-13 16:47:27.501464+00
0,,,
0,,,
0,40% mięsa? To nie karma tylko pasza.,2023-01-18 13:16:51.585716+00,2023-01-19 07:31:33.321847+00
0,,,
...,...,...,...
0,,,
0,,,
0,Zwierzęta padły po szczepionce‼️Oni chcą wszys...,2023-01-08 09:40:53.085924+00,2023-01-09 08:50:13.916888+00
0,,,


In [75]:
first_timestamp_path = working_dir / 'updated_at_0.csv'
second_timestamp_path = working_dir / 'updated_at_1.csv'

if os.path.exists(first_timestamp_path):
    os.remove(first_timestamp_path)

if os.path.exists(second_timestamp_path):
    os.remove(second_timestamp_path)

texts_to_check['updated_at_0'].to_csv(first_timestamp_path, index=False, header=False)
texts_to_check['updated_at_1'].to_csv(second_timestamp_path, index=False, header=False)